In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **I have written this notebook out of my willingness to learn LGBM. @brendanartley's [notebook](https://www.kaggle.com/brendanartley/sep-21-tab-series-lgbm-optuna) helped me a lot in learning about LGBM and writing this notebook. All credits to him.**

## **Read the data**

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df_label = df['claim']

In [ ]:
df_label

In [ ]:
df = df.drop(columns=['id', 'claim'], axis=1)

## **Check for null values**

In [ ]:
df.isnull().sum()

In [ ]:
df_mean = df.mean()

In [ ]:
df = df.fillna(df_mean)

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df.head()

In [ ]:
train_df, valid_df, train_label, valid_label = train_test_split(df, df_label, test_size = 0.2)

In [ ]:
print(train_df.shape)
print(train_label.shape)
print(valid_df.shape)
print(valid_label.shape)

## Scaling values

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_df)

In [ ]:
train_df = scaler.transform(train_df)
test_df = scaler.transform(valid_df)

In [ ]:
valid_df[valid_df < 0] = 0
valid_df[valid_df > 1] = 1

## **Training the model**

In [ ]:
import lightgbm as lgb

In [ ]:
params = {'objective':'binary',
          'num_leaves': 100,
          'n_estimators': 700,
          'max_depth': 6,
          'min_child_samples': 371,
          'learning_rate': 0.10,
          'min_data_in_leaf': 500,
          'bagging_fraction': 0.8057106694835638,
          'feature_fraction': 0.5688885590495344,
         }
model = lgb.LGBMClassifier(**params)

lgtrain = lgb.Dataset(train_df, label=train_label)
lgval = lgb.Dataset(test_df, label=valid_label)

In [ ]:
model = lgb.train(params, 
                  train_set = lgtrain,  
                  valid_sets=[lgtrain, lgval], 
                  verbose_eval=100,
                 )

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
submission_df = test_df['id'].to_frame()
test_df_np = test_df.drop(columns=['id'], axis=0).fillna(df_mean)

test_df_np = scaler.transform(test_df_np)
test_df_np[test_df_np < 0] = 0
test_df_np[test_df_np > 1] = 1

In [ ]:
submission_df['claim'] = model.predict(test_df_np)
submission_df.to_csv("submission.csv", index=False)

submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)